<a href="https://www.kaggle.com/code/nithunith/vgg-net-classification?scriptVersionId=185879605" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# General Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torch.optim import lr_scheduler
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data agumentation and normalization for training

In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#having only normalization for transformation
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#load the dataset 
dataset = datasets.ImageFolder(root='/kaggle/input/tom-and-jerry-image-classification/tom_and_jerry/tom_and_jerry/')

# Define the train-validation split ratio
train_ratio = 0.7
val_ratio = 0.3
total_size = len(dataset)
train_size = int(total_size * train_ratio)
val_size = total_size - train_size

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply transformations to the datasets
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

# Create DataLoader
dataloaders = {
    
    'train' : DataLoader(dataset=train_dataset, batch_size=32, shuffle=True),
    'val'   : DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
    
}

# Calculate dataset sizes
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

In [ ]:
#to have an idea about the encoding details
print(dataset.class_to_idx)
class_names = list(dataset.class_to_idx.keys())
print("Class names",class_names)

# Image Visualization

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
#setting the figure size
plt.figure(figsize=(15,15))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
#function call to show the image
imshow(out, title=[class_names[x] for x in classes])

# Model Training Script

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, device, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # Zero the parameter gradients
                    optimizer.zero_grad()

                    # Forward
                    # Track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # Backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # Statistics
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # Deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # Load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model


# Vizualizing the model predictions

In [ ]:
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training) # restore the model to traning state if the model was in training

# 1. Finetuning the VGG Model - Using the pre-trained for initialization and training the model

In [ ]:
#Pytorch command to list all the avaliable models
models.list_models()  

In [ ]:
model_ft = models.vgg11_bn(weights='IMAGENET1K_V1')


In [ ]:
#print the model classifier
print(model_ft.classifier)

# Modify the classifier part
# The classifier is a sequential container; the last layer is the one we need to change
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, len(class_names))

print(model_ft.classifier)

In [ ]:
#load the model to device
model_ft = model_ft.to(device)

# Modify the classifier part
# The classifier is a sequential container; the last layer is the one we need to change
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train the Model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,dataloaders,dataset_sizes,device,num_epochs=20)

# Visualize the prediction

In [ ]:
visualize_model(model_ft)

In [ ]:
torch.save(model_ft.state_dict(), 'tom_jerry.pt')

# Grad Cam

In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

target_layers = [model_ft.features[-1]]

cam = GradCAM(model=model_ft, target_layers=target_layers)

inputs, classes = next(iter(dataloaders['val']))
inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
classes = classes.to('cuda' if torch.cuda.is_available() else 'cpu')


# Generate CAM for the batch
targets = [ClassifierOutputTarget(cls.item()) for cls in classes]
grayscale_cam = cam(input_tensor=inputs, targets=targets)

# Convert to numpy and display
inputs = inputs.cpu().numpy().transpose(0, 2, 3, 1)  # Convert to HWC format
inputs = inputs * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Denormalize
inputs = np.clip(inputs, 0, 1)

# Display the images and their corresponding CAMs
fig, axes = plt.subplots(len(inputs), 2, figsize=(10, len(inputs) * 5))
for i in range(len(inputs)):
    cam_image = show_cam_on_image(inputs[i], grayscale_cam[i], use_rgb=True)
    
    axes[i, 0].imshow(inputs[i])
    axes[i, 0].set_title(f'Original Image - Class: {classes[i].item()}')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(cam_image)
    axes[i, 1].set_title(f'Grad-CAM - Class: {classes[i].item()}')
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()